In [1]:
import pandas as pd
import numpy as np 

PATH_DETAIL= r'.\input\wawa_all_detail_20230813-215105.csv'

df = pd.read_csv(PATH_DETAIL,delimiter=';')

In [2]:
# select only relevant cols
mycols=['id',
 'publicId',
 'advertType',#['AGENCY', 'PRIVATE', 'DEVELOPER_UNIT']
 'MarketType',#['secondary', 'primary'], 
 'title',
 'Price',
 'Area',
 'Rooms_num',
 'createdAt',
 'modifiedAt',
#  'features',
#  'Extras_types',
 'agency',
 'rent', 
 'condition',#['TO_RENOVATION', 'READY_TO_USE', 'TO_COMPLETION', nan]
 'ownership',#['FULL_OWNERSHIP', nan, 'LIMITED_OWNERSHIP', 'SHARE', 'USUFRUCT'],
 'Building_type',#['house', 'block', 'apartment', 'tenement', nan, 'ribbon', 'infill','loft'], 
 'Building_material',#['brick', 'concrete_plate', nan, 'other', 'reinforced_concrete','breezeblock', 'concrete', 'silikat', 'cellular_concrete', 'wood']
 'Heating',#['gas', 'urban', nan, 'boiler_room', 'other', 'electrical'],
 'Windows_type',#['wooden', nan, 'plastic', 'aluminium']
 'Build_year', 
 'Lift',
 'Floor_no',
 'Building_floors_num',

 'latitude',
 'longitude',
 'cityname',
 'districtname',
 'streetname',
 'voivodeship',
 'city_or_village',
 'district',
 'residential',]

df=df[mycols]

In [3]:
# TYPE correction
category_cols = ['condition','advertType','MarketType','ownership','Building_type','Building_material','Heating','Windows_type',]
for col in category_cols:
    df[col]=df[col].astype('category')

df['Price']=df['Price'].astype('float')
df['rent']=df['rent'].round().astype('float')

def convert_Rooms_num(val):
    if val == 'more':
        return 99.0
    else:
        return float(val)
df['Rooms_num']=df['Rooms_num'].apply(convert_Rooms_num).astype('Int64')
df['Building_floors_num']=df['Building_floors_num'].apply(convert_Rooms_num).astype('Int64')

def convert_Floor_no(val):
    if val == 'ground_floor':
        return 0
    elif val == 'garret':
        return -1
    elif val == 'cellar':
        return 99
    elif val == 'floor_higher_10':
        return 99
    elif type(val) == str:
        return float(val.replace('floor_',''))
df['Floor_no']=df['Floor_no'].apply(convert_Floor_no).astype('Int64')

df['createdAt']=pd.to_datetime(df['createdAt'],utc=True) 
df['modifiedAt']=pd.to_datetime(df['modifiedAt'],utc=True) 
df['Build_year']=df['Build_year'].astype('Int64')

df["Lift"] = np.where(df["Lift"] == "y", True, False)

In [4]:
# CALCULATE new cols
df["Price_pm"] = round(df["Price"] / df["Area"] / 1000, 2)
df["parter"] = np.where((df["Floor_no"].notna()) & (df["Floor_no"] == 0), True, False)
df["last"] = np.where(
    (df["Floor_no"].notna())
    & (df["Building_floors_num"].notna())
    & (df["Floor_no"] >= df["Building_floors_num"]),
    True,
    False,
)

def clasify_Building_type(row:pd.Series):
    
    hc1=row['Building_floors_num']<3

    bc1=row['Build_year']>1970
    bc2=row['Lift'] == True
    bc3=row['Building_material']in['concrete_plate']
    bc4=row['Building_floors_num']>6
    
    kc1=row['Build_year']<=1970

    if pd.notna(row['Building_type']):
        if row['Building_type']=='apartment':
            return 'block'
        else:
            return row['Building_type']
    else:
        if pd.notna(hc1)&hc1:
            return 'house'
        if pd.notna(bc1)&bc1:
            return 'block'
        if pd.notna(bc3)&bc3:
            return 'block'
        if pd.notna(bc4)&bc4:
            return 'block'
        if pd.notna(bc2)&bc2:
            return 'block'
        if pd.notna(kc1)&kc1:
            return 'tenement' #kamienica     
               
df['MY_Building_type']=df.apply(clasify_Building_type,axis=1)

def clasify_condition(row:pd.Series):
    
    yc1=row['Build_year']>2010
    wc1=row['Windows_type'] == 'wooden'

    if pd.notna(row['condition']):
        return row['condition']
    else:
        if pd.notna(yc1)&yc1:
            return 'READY_TO_USE'
        if pd.notna(wc1)&wc1:
            return 'TO_RENOVATION'
        
df['MY_condition']=df.apply(clasify_condition,axis=1)

In [11]:
# DROP NA
df.dropna(subset=['Price'],inplace=True)

In [16]:
# df.info()
# df.isna().sum()
# df.describe()
df.describe(exclude=np.number)


C:\Users\krawc\AppData\Local\Temp\ipykernel_5820\3103699120.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(exclude=np.number)
C:\Users\krawc\AppData\Local\Temp\ipykernel_5820\3103699120.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(exclude=np.number)


,publicId,advertType,MarketType,title,createdAt,modifiedAt,agency,condition,ownership,Building_type,...,districtname,streetname,voivodeship,city_or_village,district,residential,parter,last,MY_Building_type,MY_condition
count,7295,7295,7295,7295,7295,7295,6308,5896,5616,5220,...,7172,5447,7250,7250,7224,7224,7295,7295,7151,6478
unique,7295,3,2,6471,6957,6371,730,3,4,7,...,138,1337,1,11,18,135,2,2,6,3
top,2f1sl,AGENCY,secondary,Rewelacyjna Oferta Dla Rodziny Lub Na Inwestycję,2023-05-15 13:35:39+00:00,2023-08-12 04:57:30+00:00,TEstate,READY_TO_USE,FULL_OWNERSHIP,block,...,Mokotów,ul. Kazimierza Gierdziejewskiego,mazowieckie,Warszawa,Wola,Mirów,False,False,block,READY_TO_USE
freq,1,5209,4915,23,7,8,209,3142,5055,2534,...,637,131,7250,7224,968,319,6268,5801,5918,3653
first,NaN,NaN,NaN,NaN,2014-11-10 11:05:12+00:00,2023-07-06 14:41:43+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2023-08-08 15:05:39+00:00,2023-08-13 18:00:12+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
